In [7]:
from xgboost import XGBClassifier
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.linear_model import LinearRegression  # For linear models
from sklearn.ensemble import RandomForestClassifier  # For tree-based models
from sklearn.svm import SVC  # For support vector machines
from sklearn.neural_network import MLPClassifier  # For neural networks
from sklearn.model_selection import RandomizedSearchCV

matches_with_features = pd.read_csv('tennis_matches_enriched.csv', parse_dates=['date'])


predictors = [
    "playerCode", "opponentCode",
    "Rank_1", "Rank_2", "Pts_1", "Pts_2",
    "h2h_matches", "h2h_win_pct", "h2h_surface_matches", "h2h_surface_win_pct",
    "recent_matches_p1", "recent_win_pct_p1", "recent_avg_opp_rank_p1",
    "recent_matches_p2", "recent_win_pct_p2", "recent_avg_opp_rank_p2",
    "win_pct_surface_p1", "win_pct_surface_p2",
    "series_level", "round_num", "best_of",
    "implied_prob_p1", "implied_prob_p2",
]

# Split data - Updated version
train = matches_with_features[
    (matches_with_features['date'] >= '2000-01-03') & 
    (matches_with_features['date'] <= '2022-12-31')  # Train on first 23 years
]

test = matches_with_features[
    (matches_with_features['date'] >= '2023-01-01') & 
    (matches_with_features['date'] <= '2024-06-29')  # Test on most recent 1.5 years
]


param_grid = {
    'n_estimators': [300, 500, 700],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'min_child_weight': [1, 5, 10]
}

search = RandomizedSearchCV(
    estimator=XGBClassifier(),
    param_distributions=param_grid,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    random_state=1
)

search.fit(train[predictors], train["target"])
print(f"Best params: {search.best_params_}")

"""
model = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8)
model.fit(train[predictors], train["target"])"""

#print(f"n_estimators={500}: Test accuracy = {model.score(test[predictors], test['target']):.3f}")

KeyboardInterrupt: 